In [1]:
from tqdm.auto import tqdm 
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nf
from netCDF4 import Dataset
%matplotlib inline
import glob
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [2]:
datapath="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/Data/ECMWF/ERA5_25kmx3hr/"
path="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/"
output="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/create_ts/outputs/"
target="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/create_ts/outputs/targets/"

In [3]:
!ls /work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/

2001_wp_storms.csv  2005_wp_storms.csv	2009_wp_storms.csv  all
2002_wp_storms.csv  2006_wp_storms.csv	2010_wp_storms.csv  nio
2003_wp_storms.csv  2007_wp_storms.csv	2020_nh_storms.csv  wp
2004_wp_storms.csv  2008_wp_storms.csv	2021_nh_storms.csv


### Read tracks

In [4]:
track = sorted(glob.glob(path+'*2009*.csv'))

In [5]:
tracksDF = pd.read_csv(track[0])

In [6]:
tracksDF['name'].unique()

array(['KUJIRA', 'CHAN-HOM', 'LINFA', 'GONI', 'MORAKOT', 'ETAU', 'VAMCO',
       'KROVANH', 'DUJUAN', 'CHOI-WAN', 'PARMA', 'MELOR', 'NEPARTAK',
       'LUPIT', 'MIRINAE', 'NIDA'], dtype=object)

### Extract time series

In [7]:
def largearea_withpres(dataset=None,invar=None,indices=None,tc_irad=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    rgspt=16
    var_out=np.zeros((len(indices),len(dataset.plev.data),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        for ip in range(len(dm1.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out  

In [8]:
##########function############
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    rgspt=16
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out

def largearea_withpres(dataset=None,invar=None,indices=None,tc_irad=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    rgspt=16
    var_out=np.zeros((len(indices),len(dataset.plev.data),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        for ip in range(len(dm1.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out  

In [9]:
stormnames = list(tracksDF['name'].unique())

In [10]:
[str(tracksDF[tracksDF['name']==stormnames[0]].time[i]).split(':')[0] for i in range(len(tracksDF[tracksDF['name']==stormnames[0]].time))]

['2009-04-30',
 '2009-04-30',
 '2009-05-01',
 '2009-05-01',
 '2009-05-01',
 '2009-05-01',
 '2009-05-01',
 '2009-05-01',
 '2009-05-01',
 '2009-05-01',
 '2009-05-02',
 '2009-05-02',
 '2009-05-02',
 '2009-05-02',
 '2009-05-02',
 '2009-05-02',
 '2009-05-02',
 '2009-05-02',
 '2009-05-03',
 '2009-05-03',
 '2009-05-03',
 '2009-05-03',
 '2009-05-03',
 '2009-05-03',
 '2009-05-03',
 '2009-05-03',
 '2009-05-04',
 '2009-05-04',
 '2009-05-04',
 '2009-05-04',
 '2009-05-04',
 '2009-05-04',
 '2009-05-04',
 '2009-05-04',
 '2009-05-05',
 '2009-05-05',
 '2009-05-05',
 '2009-05-05',
 '2009-05-05',
 '2009-05-05',
 '2009-05-05',
 '2009-05-05',
 '2009-05-06',
 '2009-05-06',
 '2009-05-06',
 '2009-05-06',
 '2009-05-06',
 '2009-05-06',
 '2009-05-06',
 '2009-05-06',
 '2009-05-07',
 '2009-05-07',
 '2009-05-07',
 '2009-05-07',
 '2009-05-07',
 '2009-05-07',
 '2009-05-07',
 '2009-05-07',
 '2009-05-08',
 '2009-05-08',
 '2009-05-08',
 '2009-05-08',
 '2009-05-08',
 '2009-05-08',
 '2009-05-08',
 '2009-05-08',
 '2009-05-

In [11]:
#for dm1 files functions
def make_timeseries_step1(TCname=None,era5_date=None,era5_hour=None):
    track=tracksDF[tracksDF['name']==TCname].reset_index()
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    return pos,indices_store,tc_irad

def make_timeseries_step3(dm1=None,pvarname=None,pvardict=None,indices_store=None):
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    return ts_pdict

In [12]:
def readyear_automatic(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/radvars_'+str(year)+'.nc')
    #tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/nio/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,era5_date,era5_hour

dm2,era5_date,era5_hour = readyear_automatic(2009)

In [13]:
TCrl_ts = []
for TCname in tqdm(stormnames):
    ###########################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    svarname = ['cape','inst_10m_wnd_gst','inst_moisture_flux','inst_ssh_flux','surfmean_swr_flux','surfmean_lhf',\
           'surfmean_lwr_flux','surfmean_shf','dwnwrdmean_swr_flux','topmean_lwr_flux','topmean_swr_flux',\
           'vimean_moisture_div','surf_lhf','surf_shf','tot_suprcool_liqwtr','tot_wtr_vpr']
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    #############################################################################################

    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCrl_ts.append(tsdict)

  0%|          | 0/16 [00:00<?, ?it/s]

In [14]:
storeTCdicts = {}
for ind in range(len(TCrl_ts)):
    storeTCdicts[stormnames[ind]] = pd.DataFrame.from_dict(TCrl_ts[ind])

In [15]:
for ind,obj in enumerate(stormnames):
    storeTCdicts[obj].to_csv(output+'2009_radvar_wpac_'+str(obj)+'.csv')

In [16]:
pd.DataFrame.from_dict(TCrl_ts[3])

,cape,inst_10m_wnd_gst,inst_moisture_flux,inst_ssh_flux,surfmean_swr_flux,surfmean_lhf,surfmean_lwr_flux,surfmean_shf,dwnwrdmean_swr_flux,topmean_lwr_flux,topmean_swr_flux,vimean_moisture_div,surf_lhf,surf_shf,tot_suprcool_liqwtr,tot_wtr_vpr
0,239.004883,10.388416,-0.000061,-15.733154,0.000000,-142.741882,-31.905182,-16.616272,0.000000,-151.232422,0.000000,-0.000053,-513871.000,-59816.25,0.062388,66.840430
1,353.062500,10.298082,-0.000061,-17.429382,0.000000,-137.203003,-35.186035,-15.780762,0.000000,-144.543274,0.000000,-0.000081,-493928.125,-56813.50,0.043180,68.075485
2,438.791016,10.022598,-0.000056,-19.447388,0.000000,-133.854797,-33.136688,-18.230713,0.000000,-135.367188,0.000000,-0.000502,-481877.750,-65634.50,0.062830,68.349380
3,537.225586,11.021526,-0.000059,-19.556213,0.000000,-143.641479,-33.366577,-20.231812,0.000000,-140.527557,0.000000,-0.000711,-517111.250,-72834.00,0.077646,68.637733
4,461.005371,10.888556,-0.000051,-14.554993,62.830933,-141.330627,-34.563263,-18.625244,662.288208,-137.002502,348.938721,-0.000655,-508790.000,-67052.00,0.071162,71.056274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,845.081055,8.321689,-0.000029,0.938965,0.000000,-83.707336,-33.956970,-0.830078,0.035767,-259.637756,0.022949,-0.000063,-301346.000,-2991.25,0.006271,67.968315
73,959.385742,8.897865,-0.000028,0.721375,0.000000,-78.474060,-32.807587,-0.852661,0.000000,-239.428650,0.000000,-0.000086,-282506.000,-3070.25,0.010756,67.028404
74,1086.899414,9.283963,-0.000028,-1.019653,0.000000,-78.255249,-27.394897,-2.069946,0.000000,-208.033783,0.000000,-0.000189,-281717.250,-7449.00,0.006639,66.561859
75,1115.286133,7.997408,-0.000026,-3.498352,0.000000,-66.133728,-29.600922,-3.098267,0.000000,-210.571594,0.000000,-0.000173,-238084.500,-11153.50,0.006822,67.499428


In [20]:
def readyear_automatic(year=None):
    dm2 = xr.open_mfdataset([datapath+'/prate/prates_'+str(year)+'.nc',datapath+'/mslp/mslp_'+str(year)+'.nc',datapath+'/sst/sst_'+str(year)+'.nc'])
    #tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/nio/*_'+str(year)+'*'))
    #era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    #era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2#,era5_date,era5_hour

dm2 = readyear_automatic(2009)

In [21]:
TCpr_ts = []
for TCname in tqdm(stormnames):
    ###########################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]

    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['mn_mslp','conv_rrate','ls_rrate','mn_conv_prate','mn_ls_prate','mn_tot_prate','sst']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCpr_ts.append(tsdict)

  0%|          | 0/16 [00:00<?, ?it/s]

In [22]:
storeTCdicts = {}
for ind in range(len(TCrl_ts)):
    storeTCdicts[stormnames[ind]] = pd.DataFrame.from_dict(TCpr_ts[ind])

In [23]:
for ind,obj in enumerate(stormnames):
    storeTCdicts[obj].to_csv(output+'2009_presstvar_wpac_'+str(obj)+'.csv')

In [24]:
pd.DataFrame.from_dict(TCpr_ts[3])

,mn_mslp,conv_rrate,ls_rrate,mn_conv_prate,mn_ls_prate,mn_tot_prate,sst
0,99682.299805,0.000089,0.000121,0.000112,0.000137,0.000249,302.673252
1,99669.466797,0.000139,0.000129,0.000127,0.000128,0.000255,302.709167
2,99462.054688,0.000437,0.000130,0.000422,0.000139,0.000562,302.709019
3,99425.217773,0.000456,0.000279,0.000463,0.000322,0.000784,302.660847
4,99596.848633,0.000429,0.000199,0.000447,0.000236,0.000683,302.355957
...,...,...,...,...,...,...,...
72,99476.551758,0.000021,0.000014,0.000014,0.000013,0.000027,301.697502
73,99636.251953,0.000008,0.000020,0.000009,0.000016,0.000025,301.652187
74,99569.784180,0.000055,0.000037,0.000038,0.000034,0.000072,301.627345
75,99624.415039,0.000071,0.000033,0.000045,0.000033,0.000078,301.602296


In [25]:
def readyear_automatic(year=None):
    dm2=xr.open_dataset(datapath+'/slev_vars/svars_'+str(year)+'.nc') #tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/nio/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,era5_date,era5_hour

dm2,era5_date,era5_hour = readyear_automatic(2009)

In [26]:
TCsl_ts = []
for TCname in tqdm(stormnames):
    ###########################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['u10','v10','2mdewtmp','2mtmp','conv_ppt','tot_cld_ice',\
           'tot_cldwtr','tot_cld_rain','vi_div_cld_froz_wtr','vi_div_cld_liq_wtr','vi_div_gpot_flux',\
           'vi_div_ke_flux','vi_div_mass_flux','vi_div_moisture_flux','vi_div_olr_flux','vi_div_tot_enrgy_flux',\
           'vi_ke','vi_pe_inte','vi_pe_ie_latentenrgy','vi_temp','vi_olr','vi_tot_enrgy','vi_moisture_div']
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in tqdm(enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCsl_ts.append(tsdict)

  0%|          | 0/16 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [27]:
storeTCdicts = {}
for ind in range(len(TCrl_ts)):
    storeTCdicts[stormnames[ind]] = pd.DataFrame.from_dict(TCsl_ts[ind])

In [28]:
for ind,obj in enumerate(stormnames):
    storeTCdicts[obj].to_csv(output+'2009_slvars_wpac_'+str(obj)+'.csv')

In [29]:
#for dm1 files functions
def make_timeseries_step1(TCname=None,era5_date=None,era5_hour=None):
    track=tracksDF[tracksDF['name']==TCname].reset_index()
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    return pos,indices_store,tc_irad

def make_timeseries_step3(dm1=None,pvarname=None,pvardict=None,indices_store=None):
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    return ts_pdict

In [30]:
dm1=xr.open_dataset(datapath+'/vor/vor_2009.nc')

In [31]:
#For vars with 21 vertical levels vorticity humidity file##################
TCv_ts = []

for TCname in tqdm(stormnames):
    ############################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['vor']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    #############################################################################################
    TCv_ts.append(ts_pdict)

  0%|          | 0/16 [00:00<?, ?it/s]

In [32]:
dm1=xr.open_dataset(datapath+'/rhum/rhum_2009.nc')

In [33]:
TCrh_ts = []

for TCname in tqdm(stormnames):
    ############################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['rhum']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    #############################################################################################
    TCrh_ts.append(ts_pdict)

  0%|          | 0/16 [00:00<?, ?it/s]

In [34]:
dm1=xr.open_dataset(datapath+'/pvor/pvor_2009.nc')

In [35]:
#potential vorticity
TCp2_ts = []

for TCname in tqdm(stormnames):
    ############################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['pvor']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCp2_ts.append(ts_pdict)

  0%|          | 0/16 [00:00<?, ?it/s]

In [36]:
dm1=xr.open_dataset(datapath+'/div/div_2009.nc')

In [37]:
#divergence
TCd_ts = []
for TCname in tqdm(stormnames):
    ############################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['div']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()

    #############################################################################################
    TCd_ts.append(ts_pdict)

  0%|          | 0/16 [00:00<?, ?it/s]

In [38]:
pilv=[str(int(obj)) for obj in dm1.plev.data/100]

In [39]:
pilv

['50',
 '100',
 '200',
 '250',
 '300',
 '400',
 '500',
 '600',
 '700',
 '800',
 '850',
 '925',
 '1000']

In [40]:
dm1=xr.open_dataset(datapath+'/gpot/gpot_2009.nc')

In [41]:
#geopotential
TCgp_ts = []
for TCname in tqdm(stormnames):
    ############################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['gpot']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCgp_ts.append(ts_pdict)

  0%|          | 0/16 [00:00<?, ?it/s]

In [42]:
dm1=xr.open_dataset(datapath+'/tlevs/tlevs_2009.nc')

In [43]:
#temperature levels
TCtl_ts = []
for TCname in tqdm(stormnames):
    ############################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['temp']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCtl_ts.append(ts_pdict)

  0%|          | 0/16 [00:00<?, ?it/s]

In [44]:
dm1=xr.open_dataset(datapath+'/vvel/w_wnd_2009.nc')

In [45]:
#Vertical velocity
TCvv_ts = []
for TCname in tqdm(stormnames):
    ###########################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['vvel']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    ts_pdict = make_timeseries_step3(dm1=dm1,pvarname=pvarname,pvardict=pvardict,indices_store=indices_store)
    #############################################################################################
    TCvv_ts.append(ts_pdict)

  0%|          | 0/16 [00:00<?, ?it/s]

In [46]:
plv = [str(int(obj)) for obj in dm1.plev.data/100]

In [47]:
dx=xr.open_dataset(datapath+'/tlevs/tlevs_2009.nc')

In [48]:
plev=[str(int(obj)) for obj in dx.plev.data/100]

In [49]:
plv

['10',
 '20',
 '30',
 '50',
 '70',
 '100',
 '150',
 '200',
 '250',
 '300',
 '400',
 '500',
 '600',
 '700',
 '850',
 '925',
 '975',
 '1000']

In [50]:
len(plv),len(pilv),len(plev),TCgp_ts[0]['gpot'].shape

(18, 13, 21, (101, 21))

In [51]:
def create_pandadict(divdict=None,vordict=None,rhdict=None,gpotdict=None,tempdict=None,vveldict=None,divlv=pilv,vorlv=plev,\
                     rhlev=plev,gpotlv=plev,templv=plev,vvellv=plv):
    tempdivdict = {'div_'+divlv[i]:divdict['div'][:,i] for i in range(divdict['div'].shape[1])}
    tempvordict = {'vor_'+vorlv[i]:vordict['vor'][:,i] for i in range(vordict['vor'].shape[1])}
    temprhdict = {'rh_'+rhlev[i]:rhdict['rhum'][:,i] for i in range(rhdict['rhum'].shape[1])}
    tempgpotdict = {'gpot_'+gpotlv[i]:gpotdict['gpot'][:,i] for i in range(gpotdict['gpot'].shape[1])}
    temptempdict = {'temp_'+templv[i]:tempdict['temp'][:,i] for i in range(tempdict['temp'].shape[1])}
    tempvveldict = {'vvel_'+vvellv[i]:vveldict['vvel'][:,i] for i in range(vveldict['vvel'].shape[1])}   
    dict1 = {**tempdivdict,**tempvordict} # change this later
    dict2 = {**temprhdict,**tempgpotdict}
    dict3 = {**temptempdict,**tempvveldict}
    combine1={**dict1,**dict2} # change this later
    alldict = {**combine1,**dict3} # change this later
    #alldict= {**dict2,**dict3}
    return alldict

In [52]:
storeTCdicts = {}
for ind in range(len(TCp2_ts)):
    storeTCdicts[stormnames[ind]] = pd.DataFrame.from_dict(create_pandadict(divdict=TCd_ts[ind],vordict=TCv_ts[ind],\
                                                                            rhdict=TCrh_ts[ind],gpotdict=TCgp_ts[ind],\
                                                                            tempdict=TCtl_ts[ind],vveldict=TCvv_ts[ind]))

In [53]:
for ind,obj in enumerate(stormnames):
    storeTCdicts[obj].to_csv(output+'2009_innerlevs_wpac_'+str(obj)+'.csv')

In [90]:
#TCp_ts[0]['vort']stands for the vorticity of the first storm and vort again contains 21 pressure levels.
#preslv = [str(int(obj)) for obj in dm1.plev.data/100]

tempvortdict = {'vort_'+plev[i]:TCv_ts[0]['vort'][:,i] for i in range(21)} # add this later
temprhumdict = {'rhum_'+plev[i]:TCv_ts[0]['rhum'][:,i] for i in range(21)} # add this later
temppvordict = {'pvor_'+preslv[i]:TCp2_ts[0]['pvor'][:,i] for i in range(18)}
tempgpotdict = {'gpot_'+plv[i]:TCgp_ts[0]['gpot'][:,i] for i in range(20)}
temptempdict = {'temp_'+plev[i]:TCtl_ts[0]['temp'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[0]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict2 = {**temppvordict,**tempgpotdict}
dict3 = {**temptempdict,**tempvveldict}
combine1={**dict1,**dict2}
alldict = {**combine1,**dict3}
#pts2020_amphan=pd.DataFrame.from_dict(plvdict)

KeyError: 'vort'

In [31]:
plev_2021=pd.DataFrame.from_dict(alldict)

In [67]:
ts_gul21=pd.concat([div_21,eqt_21,sl2021,rad2021,pr2021,plev_2021], axis=1, join='inner')

In [68]:
ts_gul21

,div_50,div_100,div_200,div_250,div_300,div_400,div_500,div_600,div_700,div_800,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,-1.470139e-05,-2.004806e-05,0.000018,1.717859e-05,1.207291e-05,8.906121e-06,-1.034816e-06,7.125200e-07,-3.069406e-06,-4.832400e-06,...,-0.179325,-0.259124,-0.360181,-0.382818,-0.385455,-0.363965,-0.295342,-0.182406,-0.074252,-0.017509
1,-2.488989e-05,-5.881069e-06,0.000022,1.001521e-05,7.239578e-06,-2.546585e-06,2.435117e-06,-5.611917e-06,-5.518668e-07,-1.899782e-06,...,-0.201773,-0.238356,-0.244928,-0.229633,-0.239007,-0.181803,-0.169252,-0.119534,-0.056306,-0.018626
2,-5.774957e-06,-1.693203e-06,0.000016,3.927038e-06,-8.301693e-06,-1.514552e-05,-1.117820e-06,2.739776e-06,-5.193695e-06,6.410002e-06,...,-0.151920,-0.137368,-0.016568,0.070683,0.044295,0.076153,0.026839,-0.005580,0.007360,0.018210
3,-4.540430e-06,5.546259e-06,0.000010,9.460666e-06,-8.159375e-06,-6.259594e-06,-4.779082e-06,1.751061e-06,-1.289300e-07,5.345559e-06,...,-0.156391,-0.158884,-0.086677,-0.047792,-0.028524,-0.031328,-0.089968,-0.073317,-0.020160,0.013664
4,1.277926e-05,2.870103e-06,0.000009,5.408539e-06,-6.243528e-06,6.831135e-06,1.678709e-07,1.325330e-06,7.220078e-07,1.140637e-06,...,-0.165562,-0.165935,-0.179049,-0.213646,-0.220336,-0.232739,-0.245793,-0.176871,-0.078091,-0.024020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,-3.872090e-06,-4.878500e-06,0.000037,4.795962e-06,8.417992e-07,4.533329e-06,-4.068715e-07,-1.222652e-05,-2.529082e-05,-1.379685e-05,...,-0.332974,-0.345632,-0.376871,-0.417450,-0.363439,-0.170226,0.073287,0.059968,0.022831,0.031646
77,1.046498e-05,-6.112386e-07,0.000041,8.770730e-07,-3.539375e-06,3.801601e-06,-2.524524e-06,-1.678348e-05,-3.209530e-05,-1.239870e-05,...,-0.463555,-0.455450,-0.437432,-0.476495,-0.402014,-0.120978,0.123721,0.091201,0.027067,0.026226
78,-3.096648e-07,-5.463196e-06,0.000052,1.734338e-05,-3.813999e-06,3.948808e-06,-9.298674e-06,-3.250362e-05,-2.863479e-05,-9.745010e-06,...,-0.633262,-0.663047,-0.619403,-0.636304,-0.430235,-0.075319,0.126537,0.079193,0.016556,0.001574
79,3.144261e-06,-1.742784e-05,0.000044,1.299020e-05,-1.123449e-05,-4.185305e-06,-1.553120e-05,-2.349971e-05,-1.744000e-05,4.005851e-07,...,-0.388316,-0.391463,-0.256868,-0.165840,0.070560,0.315451,0.356788,0.221411,0.084055,0.041359


In [69]:
ts_gul21.to_csv(output+'inner/inner_nio_20210924.csv')